In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import napari
from PIL import Image
import av
import math

In [ ]:
# methods to shorten videos
import subprocess
import os
import time

def cut_video(input_vid_dir, input_vid_name, start_seconds, end_seconds, clip_number, clip_path, last_or_no):
    """
    this method is super fast as it does not rely on encoding and deconding the video into wrappers and uses straight ffmpeg
    """
    
    # create name of video to be outputted
    output_name = input_vid_name[:-4] + 'short_{}.avi'.format("{:02d}".format(clip_number+1))
    output_path = os.path.join(clip_path, output_name)
    
    # create name of video to be outputted
    input_vid_path = os.path.join(input_vid_dir, input_vid_name)
    
    # format seconds into hh:mm:float(ss)
    start_seconds = time.strftime('%H:%M:%S', time.gmtime(start_seconds))
    end_seconds = time.strftime('%H:%M:%S', time.gmtime(end_seconds))
    
    # command format: 'ffmpeg -ss first_clip -i input_video_string -c copy -t second_clip output_video_string'
    
    if last_or_no == False:
        command = 'ffmpeg -ss {} -i {} -c copy -t {} {}'.format(start_seconds, input_vid_path, end_seconds, output_path)
        print(command)
    else:
        command = 'ffmpeg -ss {} -i {} -c copy {}'.format(start_seconds, input_vid_path, output_path)
        print(command)
    # this copies and clips the video. first number clips the first seconds, then clip everything that is seconds after that
    
    result = subprocess.run(command)


In [ ]:
def get_length(filename):
    """
    returns length of the video in seconds as a float
    """
    result = subprocess.run(["ffprobe", "-v", "error", "-show_entries",
                             "format=duration", "-of",
                             "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

In [85]:
def create_sliced_video(directory, video_for_slicing, division_number):
    """
    takes in video, spits out a folder containing shorter clips
    """
    video = os.path.join(directory, video_for_slicing)
    
    # create directoy for short clips
    clip_path = os.path.join(directory, video_for_slicing[:-4] + '_clips')
    os.mkdir(clip_path)
    
    # get length of video and length of clips based on the number of clips we want
    length = get_length(video)
    clip_length = length/division_number
    clip_length = math.floor(clip_length)
    
    # enumerate through clip numbers writing each one to the short clip directory
    for clip_number in range(division_number):
        
        start_seconds = clip_length*division_number
        
        if clip_number == division_number-1:
            cut_video(directory, video_for_slicing, start_seconds, clip_length, clip_number, clip_path, True)
            
        else:
            cut_video(directory, video_for_slicing, start_seconds, clip_length, clip_number, clip_path, False)

In [ ]:
create_sliced_video(r'C:\Users\BMLab21\Documents\CrabStreams', '2022-04-06.avi', 12)

ffmpeg -ss 00:00:00 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06.avi -c copy -t 00:02:09 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_clips\2022-04-06short_01.avi
ffmpeg -ss 00:00:12 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06.avi -c copy -t 00:02:09 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_clips\2022-04-06short_02.avi
ffmpeg -ss 00:00:24 -i C:\Users\BMLab21\Documents\CrabStreams\2022-04-06.avi -c copy -t 00:02:09 C:\Users\BMLab21\Documents\CrabStreams\2022-04-06_clips\2022-04-06short_03.avi


In [23]:
# inputs
video = r'C:\Users\BMLab21\Documents\CrabStreams\2022-04-06.avi'
history = 100
Learning_rate = 1.0/history
cap = cv2.VideoCapture(video)
bg_subtractor = cv2.createBackgroundSubtractorMOG2()

In [26]:
output_video_loc = cut_video(video, 30, 10)

In [6]:
ret, frame_1 = cap.read()
viewer = napari.view_image(frame_1, rgb=True) # 1.labels, 2.paint, 3.fill, 4.export as date e.g. desktop/arena shapes/2022_04_01.tif

In [5]:
arena_shape = Image.open(r'C:\Users\BMLab21\Documents\CrabStreams\2022-04-04_arenaMask.tif')
arena_mask = np.dstack([arena_shape,arena_shape,arena_shape])

In [ ]:
total_events = []
frame_n = 0
while(True):
    ret, frame = cap.read()
    if ret == True:
        if frame_n % 5000 == 0:
            print(frame_n)
        frame_n +=1
        img = np.asarray(frame) * arena_mask
        mask = bg_subtractor.apply(img, learningRate=Learning_rate) 
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        event_frame = int(np.sum(mask))
        total_events.append(event_frame)
    else:
        break

cap.release()
cv2.destroyAllWindows()
total_events = np.array(total_events)
print('frames:', frame_n)

In [ ]:
a = 0
events_p1000 = total_events[a:]
plt.plot(events_p1000)
#plt.plot(np.where(events_p1000>6000))

In [48]:
when_events = np.where(events_p1000>5000)
indexed_events = np.zeros(len(events_p1000))

for events in when_events[0]:
    indexed_events[events:events+100] = 1
    
indexed_events = np.hstack([np.zeros(1000), indexed_events[:len(events_p1000)]])

In [49]:
len(indexed_events)

9904

In [56]:
cap = cv2.VideoCapture(video)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)

result = cv2.VideoWriter(r'C:\Users\BMLab21\Desktop\background_test.avi', 
                         cv2.VideoWriter_fourcc(*'MJPG'), # fourcc is how openCV finds the codec # MJPG is the codec
                         30, size)
iteration = 0

while(iteration<len(indexed_events)-1):
    iteration+=1
    event = indexed_events[iteration]
    ret, frame = cap.read()
    if ret == True:
        if event == 1:
            result.write(frame)
    else:
        break

cap.release()
result.release()
cv2.destroyAllWindows()